<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ee879d6722a85a09ad7047ada350e78171fe051fdca2fdc09d3e5cfb2d7883c1
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-09 12:47:49
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.41 C
-------------------
Today PnL: -1.18 L (-0.83%)
Current PnL: -27.61 L (-17.61%)
CY Booked + Current PnL: -13.32 L (-8.5%)
-------------------
Total profit:  1.68 L
Total loss:  -29.29 L
-------------------
Total Booked + Current PnL: 13.66 L (10.52%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.1%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.07 L (65.79%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.68%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.66,14.46,6.06,21.40,18410.0,38376.0,303802.0,6.20,60.0,X-LC,5.70,14.0,2.08,2.15,21.70,XY25,NTT,FMCG
77,TTKPRESTIG,770.00,-0.69,-21.94,28.23,0.09,22207.0,-22111.0,78666.0,82.50,33.0,M-SC,3.34,253.0,-1.00,0.56,3.35,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-1.09,3.43,15.27,19.23,23763.0,5162.0,155622.0,21.35,67.0,L-LC,7.07,182.0,0.22,1.10,25.44,XY25,ATH,MINING
49,MASFIN,398.61,-2.26,-3.86,26.95,22.05,25387.0,-3780.0,94200.0,-16.89,46.0,H-SC,6.50,168.0,-0.15,0.67,36.93,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-1.55,-14.61,17.13,0.02,25534.0,-25499.0,149060.0,127.12,53.0,M-SC,8.06,240.0,-1.00,1.05,29.66,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,2.10,-17.44,52.03,25.51,79939.0,-32455.0,153640.0,-5.62,45.0,H-LC,1.73,50.0,-0.41,1.09,13.22,AR,ATH,ELECTRICAL
5,ASIANPAINT,3460.26,1.81,-0.84,21.97,20.95,54847.0,-2121.0,249647.0,-1.90,56.0,X-LC,12.26,36.0,-0.04,1.76,34.49,X40,ATH,PAINTS
1,ABB,7934.00,1.37,-6.61,55.87,45.57,136502.0,-17299.0,244320.0,-40.03,45.0,H-MC,4.69,121.0,-0.13,1.73,3.96,AR,NTT,ELECTRICAL
44,JCHAC,2282.00,1.36,-33.96,51.40,-0.01,43386.0,-43396.0,84409.0,17182.61,52.0,M-SC,13.73,235.0,-1.00,0.60,8.03,OX40N,BTT,AC
32,HCLTECH,1922.01,1.04,7.97,15.52,24.73,40541.0,19293.0,261217.0,12.76,56.0,X-LC,6.25,13.0,0.48,1.85,24.43,X40,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,-5.45,-33.69,110.53,39.61,75127.0,-34530.0,67970.0,-55.07,49.0,H-SC,30.29,136.0,-0.46,0.48,10.02,AR,ATH,MISC
37,IEX,219.00,-4.26,-2.48,52.41,48.63,102193.0,-4967.0,194987.0,-35.06,53.0,H-SC,13.58,137.0,-0.05,1.38,8.85,XR,NTT,MISC
76,TRIDENT,48.00,-3.98,-30.31,84.40,28.51,54264.0,-27960.0,64294.0,-17.44,37.0,M-SC,7.45,226.0,-0.52,0.45,10.02,XR,NTT,TEXTILES
4,ALKYLAMINE,4546.37,-3.77,-34.12,193.90,93.63,128976.0,-34446.0,66517.0,-29.39,40.0,H-SC,16.67,143.0,-0.27,0.47,0.56,SR,ATH,CHEMICALS
61,ROUTE,2227.21,-3.36,-49.11,232.17,69.05,157226.0,-65346.0,67720.0,-58.98,41.0,H-SC,18.89,140.0,-0.42,0.48,4.44,SR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-1.22,-0.69,21.58,20.74,43542.0,-1402.0,201768.0,31.15,54.0,M-MC,5.4,189.0,-0.03,1.43,9.85,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.08,-2.82,113.94,107.91,163487.0,-4159.0,143485.0,-20.96,50.0,M-SC,10.12,220.0,-0.03,1.01,6.99,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.55,-14.61,17.13,0.02,25534.0,-25499.0,149060.0,127.12,53.0,M-SC,8.06,240.0,-1.00,1.05,29.66,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.07,-17.67,60.37,32.03,65524.0,-23297.0,108538.0,-10.83,65.0,H-SC,6.89,157.0,-0.36,0.77,21.57,OX40N,NTT,IT
47,KANSAINER,340.00,0.70,-21.16,43.92,13.47,93381.0,-57051.0,212616.0,-67.53,52.0,H-SC,4.53,159.0,-0.61,1.50,8.11,XY24,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.69,-21.94,28.23,0.09,22207.0,-22111.0,78666.0,82.50,33.0,M-SC,3.34,253.0,-1.00,0.56,3.35,OX40N,NTT,DURABLES


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-1.09,3.43,15.27,19.23,23763.0,5162.0,155622.0,21.35,67.0,L-LC,7.07,182.0,0.22,1.10,25.44,XY25,ATH,MINING
84,WIPRO,420.00,-0.06,7.48,60.27,72.27,109767.0,12677.0,182125.0,-7.89,50.0,M-LC,6.17,101.0,0.12,1.29,14.12,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-0.06,7.48,60.27,72.27,109767.0,12677.0,182125.0,-7.89,50.0,M-LC,6.17,101.0,0.12,1.29,14.12,XR,NTT,IT
25,FINCABLES,1641.55,-1.08,-2.82,113.94,107.91,163487.0,-4159.0,143485.0,-20.96,50.0,M-SC,10.12,220.0,-0.03,1.01,6.99,OX40N,ATH,CABLES
37,IEX,219.00,-4.26,-2.48,52.41,48.63,102193.0,-4967.0,194987.0,-35.06,53.0,H-SC,13.58,137.0,-0.05,1.38,8.85,XR,NTT,MISC
38,INDIAMART,4810.62,0.09,-6.88,122.00,106.72,140112.0,-8490.0,114846.0,-53.87,37.0,H-SC,7.21,138.0,-0.06,0.81,14.88,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-1.23,-5.48,43.49,35.63,86151.0,-11481.0,198093.0,-18.95,36.0,H-MC,3.19,119.0,-0.13,1.40,10.28,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.10,-16.45,32.59,10.78,65437.0,-39528.0,200790.0,-50.14,13.0,X-LC,5.77,1.0,-0.60,1.42,0.0,X40,NTT,FMCG
7,AWL,485.00,-2.15,-27.74,117.55,57.19,277004.0,-90479.0,235648.0,-65.80,23.0,X-MC,9.27,58.0,-0.33,1.67,0.0,XY24,NTT,FMCG
52,PAGEIND,51605.08,-1.06,-12.94,50.04,30.62,103268.0,-30684.0,206370.0,-38.08,26.0,X-MC,16.42,57.0,-0.30,1.46,0.0,X40,ATH,APPARELS
8,BAJAJHFL,181.50,-2.00,-20.33,95.14,55.47,184413.0,-49453.0,193833.0,-29.32,31.0,X-MC,16.36,63.0,-0.27,1.37,0.0,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-0.39,-40.43,126.96,35.20,97315.0,-52020.0,76650.0,0.51,31.0,X-SC,17.34,93.0,-0.53,0.54,0.0,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.97,-1.78,22.95,20.77,52781.0,-4163.0,229981.0,-10.33,31.0,X-MC,0.97,70.0,-0.08,1.63,5.02,X40N,NTT,BREWERIES
45,JIOFIN,387.00,-0.85,-6.38,32.97,24.48,82525.0,-17062.0,250303.0,-7.99,41.0,X-LC,1.71,37.0,-0.21,1.77,45.08,XY24,BTT,FINANCE
65,SIEMENS,4671.50,2.10,-17.44,52.03,25.51,79939.0,-32455.0,153640.0,-5.62,45.0,H-LC,1.73,50.0,-0.41,1.09,13.22,AR,ATH,ELECTRICAL
81,VBL,671.64,-1.72,-0.56,36.43,35.67,114422.0,-1755.0,314087.0,-12.33,55.0,X-LC,2.22,5.0,-0.02,2.22,13.22,X40N,ATH,FMCG
20,DABUR,735.00,0.54,2.11,40.74,43.70,103191.0,5223.0,253291.0,-6.31,66.0,X-MC,2.24,73.0,0.05,1.79,17.47,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.52,-27.95,127.83,64.15,219152.0,-66511.0,171440.0,-58.06,35.0,X-SC,6.26,84.0,-0.30,1.21,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-2.15,-27.74,117.55,57.19,277004.0,-90479.0,235648.0,-65.80,23.0,X-MC,9.27,58.0,-0.33,1.67,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.39,-40.43,126.96,35.20,97315.0,-52020.0,76650.0,0.51,31.0,X-SC,17.34,93.0,-0.53,0.54,0.0,X40,NTT,FOOTWEAR
8,BAJAJHFL,181.50,-2.00,-20.33,95.14,55.47,184413.0,-49453.0,193833.0,-29.32,31.0,X-MC,16.36,63.0,-0.27,1.37,0.0,X40N,ATH,FINANCE
52,PAGEIND,51605.08,-1.06,-12.94,50.04,30.62,103268.0,-30684.0,206370.0,-38.08,26.0,X-MC,16.42,57.0,-0.30,1.46,0.0,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.88,-27.90,98.13,42.84,45990.0,-18140.0,46866.0,-52.62,50.0,X-SC,27.03,83.0,-0.39,0.33,6.03,XY24,NTT,MISC
58,RELAXO,1176.00,-0.51,-48.47,195.59,52.31,146293.0,-70364.0,74796.0,-45.12,39.0,X-SC,7.84,92.0,-0.48,0.53,0.12,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.39,-40.43,126.96,35.20,97315.0,-52020.0,76650.0,0.51,31.0,X-SC,17.34,93.0,-0.53,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-1.23,-0.96,25.27,24.08,31774.0,-1213.0,125737.0,-9.02,48.0,X-SC,6.27,91.0,-0.04,0.89,18.47,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.98,-17.97,74.72,43.32,99826.0,-29272.0,133600.0,-29.54,42.0,X-SC,7.88,90.0,-0.29,0.94,2.58,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-0.10,-16.45,32.59,10.78,65437.0,-39528.0,200790.0,-50.14,13.0,X-LC,5.77,1.0,-0.60,1.42,0.00,X40,NTT,FMCG
75,TMPV,600.00,-1.20,-27.21,68.94,22.98,113361.0,-61464.0,164434.0,-24.30,42.0,X-LC,4.13,2.0,-0.54,1.16,3.42,XY24,NTT,AUTO
73,TCS,4389.96,-0.12,-12.01,37.19,20.71,123768.0,-45437.0,332800.0,-24.27,46.0,X-LC,2.47,3.0,-0.37,2.35,11.20,X40,ATH,IT
81,VBL,671.64,-1.72,-0.56,36.43,35.67,114422.0,-1755.0,314087.0,-12.33,55.0,X-LC,2.22,5.0,-0.02,2.22,13.22,X40N,ATH,FMCG
41,INFY,1972.00,-0.16,9.65,22.43,34.24,76953.0,30191.0,343079.0,-14.57,49.0,X-LC,7.66,6.0,0.39,2.43,18.73,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.06,-34.74,103.62,32.89,52212.0,-26818.0,50388.0,-722.51,58.0,L-MC,7.73,259.0,-0.51,0.36,38.76,XR,NTT,BANKS
13,BSOFT,831.70,-1.08,-22.59,94.78,50.79,103201.0,-31768.0,108885.0,-0.66,50.0,H-SC,10.67,171.0,-0.31,0.77,26.78,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.55,-14.61,17.13,0.02,25534.0,-25499.0,149060.0,127.12,53.0,M-SC,8.06,240.0,-1.00,1.05,29.66,OX40N,NTT,PAINTS
69,SURYODAY,216.00,0.46,-17.03,48.29,23.03,71746.0,-30498.0,148573.0,62.28,60.0,H-SC,4.15,165.0,-0.43,1.05,47.41,XR,NTT,BANKS
17,COALINDIA,484.83,-1.09,3.43,15.27,19.23,23763.0,5162.0,155622.0,21.35,67.0,L-LC,7.07,182.0,0.22,1.10,25.44,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-0.96,-29.54,60.23,12.90,92731.0,-64534.0,153962.0,127.16,62.0,H-SC,10.23,173.0,-0.70,1.09,28.02,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,-0.07,-17.67,60.37,32.03,65524.0,-23297.0,108538.0,-10.83,65.0,H-SC,6.89,157.0,-0.36,0.77,21.57,OX40N,NTT,IT
17,COALINDIA,484.83,-1.09,3.43,15.27,19.23,23763.0,5162.0,155622.0,21.35,67.0,L-LC,7.07,182.0,0.22,1.10,25.44,XY25,ATH,MINING
36,ICICIPRULI,790.00,0.86,13.72,14.96,30.73,33514.0,27022.0,224027.0,-12.89,72.0,X-MC,6.62,79.0,0.81,1.58,28.24,X40,ATH,INSURANCE
82,VOLTAS,1856.00,0.08,15.02,26.23,45.20,57849.0,28803.0,220545.0,3.76,63.0,X-MC,4.92,80.0,0.50,1.56,22.84,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.32
1,25,44.36
2,50,76.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.74
MC    30.74
LC    26.54
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.35
X40      23.97
X40N     12.76
XY25     10.72
XR        9.01
AR        8.58
OX40N     7.68
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.34
X-LC    23.06
H-SC    22.88
M-SC    11.27
X-SC     7.81
H-MC     4.61
M-MC     1.43
M-LC     1.29
L-LC     1.10
H-LC     1.09
L-SC     0.78
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.08,-9.05,44.83
IT,12.89,-17.41,76.62
FINANCE,11.06,-14.12,62.02
MISC,7.10,-29.40,82.33
ELECTRICAL,5.95,-11.47,52.30
PAINTS,5.78,-12.70,29.44
INSURANCE,4.74,-1.15,35.76
PHARMA,3.93,-5.71,38.99
AUTO,2.78,-32.91,78.99


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3243186.0,21
AR,1358872.0,10
XR,1218288.0,12
X40,1128745.0,15
X40N,937119.0,9
OX40N,706831.0,10
XY25,427636.0,7
SR,286202.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3559938.0,24
X-MC,1590311.0,16
M-SC,1463429.0,15
X-LC,962656.0,13
X-SC,813634.0,8
H-MC,426404.0,3
L-SC,181284.0,2
M-LC,109767.0,1
H-LC,79939.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1256921.0      6
           AR         925652.0      5
M-SC       XY24       842108.0      6
H-SC       XR         782932.0      7
X-MC       X40        513332.0      7
           XY24       447604.0      3
X-LC       X40        417992.0      6
X-MC       X40N       404918.0      4
X-SC       X40N       319297.0      3
H-SC       OX40N      308231.0      4
M-SC       OX40N      305983.0      5
X-SC       XY24       296916.0      3
H-SC       SR         286202.0      2
X-MC       XY25       224457.0      2
H-MC       AR         222653.0      2
X-LC       X40N       212904.0      2
H-MC       XY24       203751.0      1
X-SC       X40        197421.0      2
X-LC       XY24       195886.0      2
M-SC       XR         184710.0      2
X-LC       XY25       135874.0      3
M-SC       AR         130628.0      2
M-LC       XR         109767.0      1
L-SC       OX40N       92617.0      1
           XR          88667.0      1
H-LC       AR          79939.0      1
L-MC       XR          52212.0      1
M-MC       XY25        43542.0      1
L-LC       XY25        23763.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
